In [15]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np

tf.enable_eager_execution()

from preprocessing import inception_preprocessing
from riptide.data import *
import functools

ValueError: tf.enable_eager_execution must be called at program startup.

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [3]:
preprocess = functools.partial(inception_preprocessing.preprocess_image, height=224, width=224, is_training=False)

In [7]:
test_dataset = tfrecord_dataset("/data2/imagenet/tfrecords/", "validation", workers =4 , preprocess=preprocess, batch_size=64, return_dataset=False)

In [5]:
from tensorflow.python.keras.applications.mobilenet import MobileNet
with tf.device("/gpu:0"):
    model = MobileNet(weights='imagenet')

In [14]:
# test read speed
import time
start = time.time()
current = time.time()
with tf.device("/gpu:0"):
    for i, (data, label) in enumerate(tfe.Iterator(test_dataset)):
        if i %10 ==0:
            z = model(data)
            get = time.time()
            print("Step: %d, Time: %f" % (i, get - current))
            current = get
    print(current - start)

Tensor("IteratorGetNext:0", shape=(?, 224, 224, 3), dtype=float32)


NameError: name 'data' is not defined

In [8]:
def evaluate(model, iterator, k=1):
    metric = tfe.metrics.Mean()
    for i, (data, label) in enumerate(iterator):
        with tf.device("/gpu:0"):
            z = model(data)
        metric(tf.cast(tf.nn.in_top_k(z.cpu(), tf.squeeze(label.cpu())-1, k=5), tf.float32))
        if i%10 == 0:
            print(i)
    print(metric.value())

In [ ]:
evaluate(model, tfe.Iterator(test_iterator))